In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession
.builder
.appName("SparkSQLExampleApp")
.getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-C8F58JN:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1618307193136)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3582060


In [2]:
val cubed = (s: Long) => {
s * s * s
}

cubed: Long => Long = $Lambda$1991/0x0000000801590040@40843ff0


In [3]:
spark.udf.register("cubed", cubed)

res0: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$1991/0x0000000801590040@40843ff0,LongType,List(Some(class[value[0]: bigint])),Some(class[value[0]: bigint]),Some(cubed),false,true)


In [4]:
spark.range(1, 9).createOrReplaceTempView("udf_test")

In [5]:
spark.sql("select id, cubed(id) from udf_test").show()

+---+---------+
| id|cubed(id)|
+---+---------+
|  1|        1|
|  2|        8|
|  3|       27|
|  4|       64|
|  5|      125|
|  6|      216|
|  7|      343|
|  8|      512|
+---+---------+



In [10]:
val t1 = Array(35, 36, 32, 30, 40, 42, 38)
val t2 = Array(31, 32, 34, 55, 56)
val tC = Seq(t1, t2).toDF("celsius")
tC.createOrReplaceTempView("tC")

t1: Array[Int] = Array(35, 36, 32, 30, 40, 42, 38)
t2: Array[Int] = Array(31, 32, 34, 55, 56)
tC: org.apache.spark.sql.DataFrame = [celsius: array<int>]


In [11]:
tC.show()

+--------------------+
|             celsius|
+--------------------+
|[35, 36, 32, 30, ...|
|[31, 32, 34, 55, 56]|
+--------------------+



In [13]:
spark.sql("""
SELECT celsius, transform(celsius, t -> ((t * 9) div 5) + 32) as fahrenheit from tC""").show()

+--------------------+--------------------+
|             celsius|          fahrenheit|
+--------------------+--------------------+
|[35, 36, 32, 30, ...|[95, 96, 89, 86, ...|
|[31, 32, 34, 55, 56]|[87, 89, 93, 131,...|
+--------------------+--------------------+



In [14]:
spark.sql("""
SELECT celsius,
filter(celsius, t -> t > 38) as high
FROM tC
""").show()

+--------------------+--------+
|             celsius|    high|
+--------------------+--------+
|[35, 36, 32, 30, ...|[40, 42]|
|[31, 32, 34, 55, 56]|[55, 56]|
+--------------------+--------+



In [16]:
spark.sql("""SELECT celsius, exists(celsius, t -> t = 38) as threshold from tC""").show()

+--------------------+---------+
|             celsius|threshold|
+--------------------+---------+
|[35, 36, 32, 30, ...|     true|
|[31, 32, 34, 55, 56]|    false|
+--------------------+---------+



In [21]:
spark.sql("""
SELECT celsius,
reduce(
celsius,
0,
(t, acc) -> t + acc,
acc -> (acc div size(celsius) * 9 div 5) + 32
) as avgFahrenheit
FROM tC
""").show()

org.apache.spark.sql.AnalysisException:  Undefined function: 'reduce'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 3 pos 0

In [22]:
import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions._


In [23]:
val delaysPath = "C:/Users/GAME/Desktop/departuredelays.csv"
val airportsPath = "C:/Users/GAME/Desktop/airport-codes-na.txt"

delaysPath: String = C:/Users/GAME/Desktop/departuredelays.csv
airportsPath: String = C:/Users/GAME/Desktop/airport-codes-na.txt


In [25]:
val airports = spark.read
.option("header", "true")
.option("inferschema", "true")
.option("delimiter", "\t")
.text(airportsPath)
airports.createOrReplaceTempView("airports_na")

airports: org.apache.spark.sql.DataFrame = [value: string]


In [26]:
val delays = spark.read
.option("header","true")
.csv(delaysPath)
.withColumn("delay", expr("CAST(delay as INT) as delay"))
.withColumn("distance", expr("CAST(distance as INT) as distance"))
delays.createOrReplaceTempView("departureDelays")

delays: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 3 more fields]


In [27]:
val foo = delays.filter(
expr("""origin == 'SEA' AND destination == 'SFO' AND
date like '01010%' AND delay > 0"""))
foo.createOrReplaceTempView("foo")

foo: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: string, delay: int ... 3 more fields]


In [46]:
spark.sql("SELECT * FROM airports_na LIMIT 10").show()

+--------------------+
|               value|
+--------------------+
|City	State	Countr...|
|Abbotsford	BC	Can...|
| Aberdeen	SD	USA	ABR|
|  Abilene	TX	USA	ABI|
|    Akron	OH	USA	CAK|
|  Alamosa	CO	USA	ALS|
|   Albany	GA	USA	ABY|
|   Albany	NY	USA	ALB|
|Albuquerque	NM	US...|
|Alexandria	LA	USA...|
+--------------------+



In [29]:
spark.sql("SELECT * FROM departureDelays LIMIT 10").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+



In [30]:
spark.sql("SELECT * FROM foo").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [31]:
val bar = delays.union(foo)
bar.createOrReplaceTempView("bar")
bar.filter(expr("""origin == 'SEA' and destination == 'SFO' and date LIKE '01010%' AND delay > 0""")).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



bar: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: string, delay: int ... 3 more fields]


In [32]:
spark.sql("""
SELECT *
FROM bar
WHERE origin = 'SEA'
AND destination = 'SFO'
AND date LIKE '01010%'
AND delay > 0
""").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [39]:
spark.sql("""SELECT a.City, a.State, f.date, f.delay, f.distance, f.destination from foo f join airports_na a on a.IATA = f.origin""").show()

org.apache.spark.sql.AnalysisException:  cannot resolve '`a.IATA`' given input columns: [f.date, f.delay, f.destination, f.distance, f.origin, a.value]; line 1 pos 100;

org.apache.spark.sql.AnalysisException:  cannot resolve '`a.IATA`' given input columns: [f.date, f.delay, f.destination, f.distance, f.origin, a.value]; line 5 pos 3;

In [50]:
spark.sql("""CREATE TABLE departureDelaysWindow AS
SELECT origin, destination, SUM(delay) AS TotalDelays
FROM departureDelays
WHERE origin IN ('SEA', 'SFO', 'JFK')
AND destination IN ('SEA', 'SFO', 'JFK', 'DEN', 'ORD', 'LAX', 'ATL')
GROUP BY origin, destination""")

org.apache.spark.sql.AnalysisException:  Hive support is required to CREATE Hive TABLE (AS SELECT);

In [51]:
spark.sql("""
SELECT origin, destination, TotalDelays, rank
FROM (
SELECT origin, destination, TotalDelays, dense_rank()
OVER (PARTITION BY origin ORDER BY TotalDelays DESC) as rank
FROM departureDelaysWindow
) t
WHERE rank <= 3
""").show()

org.apache.spark.sql.AnalysisException:  Table or view not found: departureDelaysWindow; line 6 pos 5;

In [52]:
foo.show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [58]:
val foo2 = foo.withColumn("status", expr("CASE WHEN delay <= 10 then 'On-time' ELSE 'Delayed' END"))
foo2.show()

+--------+-----+--------+------+-----------+-------+
|    date|delay|distance|origin|destination| status|
+--------+-----+--------+------+-----------+-------+
|01010710|   31|     590|   SEA|        SFO|Delayed|
|01010955|  104|     590|   SEA|        SFO|Delayed|
|01010730|    5|     590|   SEA|        SFO|On-time|
+--------+-----+--------+------+-----------+-------+



foo2: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 4 more fields]


In [59]:
val foo3 = foo2.drop("delay")
foo3.show()

+--------+--------+------+-----------+-------+
|    date|distance|origin|destination| status|
+--------+--------+------+-----------+-------+
|01010710|     590|   SEA|        SFO|Delayed|
|01010955|     590|   SEA|        SFO|Delayed|
|01010730|     590|   SEA|        SFO|On-time|
+--------+--------+------+-----------+-------+



foo3: org.apache.spark.sql.DataFrame = [date: string, distance: int ... 3 more fields]


In [61]:
val foo4 = foo3.withColumnRenamed("status", "flight_status")
foo4.show()

+--------+--------+------+-----------+-------------+
|    date|distance|origin|destination|flight_status|
+--------+--------+------+-----------+-------------+
|01010710|     590|   SEA|        SFO|      Delayed|
|01010955|     590|   SEA|        SFO|      Delayed|
|01010730|     590|   SEA|        SFO|      On-time|
+--------+--------+------+-----------+-------------+



foo4: org.apache.spark.sql.DataFrame = [date: string, distance: int ... 3 more fields]
